In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 32
shuffle = True
seed = None

input_data_folder = "Data"
training_data_file = "Training-datasets-PredNTS.txt"
independent_data_file = "independent dataset-PredNTS.txt"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_nt(sequence, char_dict):
    
    seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i][char_dict[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [26]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup(input_seq_shape = (41, 21),
                 conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                 max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
                 lstm_decode_units = 25, ## LSTM layer parameters
                 conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                 max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
                 dense_decode_units = 128, ## Dense layer parameters
                 prob = 0.5, learn_rate = 0.0005, 
                 loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_seq_shape)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                padding = "same")(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu'
                             )(x4)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss)

    return model

In [10]:
DLNN_CORENup().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 41, 21)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 10)       2110        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 41, 10)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 13, 10)       0           ['activation[0][0]']             
                                                                                              

# Training

In [11]:
##################################################################################
##### read training file
##################################################################################
train_file_path = os.path.join(input_data_folder, training_data_file)
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
train_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
train_data.head()

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in train_data['Sequence']]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i
    
##################################################################################
##### Create OHE of sequence
##################################################################################
train_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in train_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
train_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in train_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

features = np.array(list(train_data['OHE_Sequence']))
labels = np.array(list(train_data['label']))
labels = labels.reshape((labels.shape[0], 1))

input_seq_shape = features[0].shape

folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

In [12]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    X_features = np.concatenate((fold["X_train"], 
                                 np.flip(fold["X_train"], axis=1)),
                                axis = 0
                               )
    Y_labels = np.concatenate((fold["y_train"], 
                               fold["y_train"]), 
                              axis = 0)
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(X_features.shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = X_features[index_arr], y = Y_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.
Epoch 1/100
120/120 [==============================] - ETA: 0s - loss: 0.8946
Epoch 1: val_loss improved from inf to 0.84374, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 5s 12ms/step - loss: 0.8946 - val_loss: 0.8437
Epoch 2/100
115/120 [===========================>..] - ETA: 0s - loss: 0.8095
Epoch 2: val_loss improved from 0.84374 to 0.71685, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 8ms/step - loss: 0.8066 - val_loss: 0.7169
Epoch 3/100
120/120 [==============================] - ETA: 0s - loss: 0.7195
Epoch 3: val_loss improved from 0.71685 to 0.65912, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold0.hdf5
120/120 [=====================

117/120 [============================>.] - ETA: 0s - loss: 0.4095
Epoch 59: val_loss did not improve from 0.48455
120/120 [==============================] - 1s 7ms/step - loss: 0.4101 - val_loss: 0.4930
Epoch 60/100
118/120 [============================>.] - ETA: 0s - loss: 0.4034
Epoch 60: val_loss did not improve from 0.48455
120/120 [==============================] - 1s 7ms/step - loss: 0.4042 - val_loss: 0.4876
Epoch 61/100
114/120 [===========================>..] - ETA: 0s - loss: 0.3974
Epoch 61: val_loss did not improve from 0.48455
120/120 [==============================] - 1s 8ms/step - loss: 0.3952 - val_loss: 0.4881
Epoch 62/100
117/120 [============================>.] - ETA: 0s - loss: 0.4077
Epoch 62: val_loss improved from 0.48455 to 0.48384, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 8ms/step - loss: 0.4083 - val_loss: 0.4838
Epoch 63/100
117/12

120/120 [==============================] - 1s 7ms/step - loss: 0.3708 - val_loss: 0.5182
Epoch 97/100
118/120 [============================>.] - ETA: 0s - loss: 0.3747
Epoch 97: val_loss did not improve from 0.48384
120/120 [==============================] - 1s 7ms/step - loss: 0.3740 - val_loss: 0.5086
Epoch 98/100
120/120 [==============================] - ETA: 0s - loss: 0.3786
Epoch 98: val_loss did not improve from 0.48384
120/120 [==============================] - 1s 7ms/step - loss: 0.3786 - val_loss: 0.5034
Epoch 99/100
119/120 [============================>.] - ETA: 0s - loss: 0.3777
Epoch 99: val_loss did not improve from 0.48384
120/120 [==============================] - 1s 8ms/step - loss: 0.3775 - val_loss: 0.5169
Epoch 100/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3698
Epoch 100: val_loss did not improve from 0.48384
120/120 [==============================] - 1s 7ms/step - loss: 0.3716 - val_loss: 0.5117

Train/Test model on Fold #1.
Epoch 1/100
114

120/120 [==============================] - 1s 7ms/step - loss: 0.4870 - val_loss: 0.5516
Epoch 23/100
120/120 [==============================] - ETA: 0s - loss: 0.4824
Epoch 23: val_loss improved from 0.55141 to 0.54601, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold1.hdf5
120/120 [==============================] - 1s 8ms/step - loss: 0.4824 - val_loss: 0.5460
Epoch 24/100
120/120 [==============================] - ETA: 0s - loss: 0.4790
Epoch 24: val_loss did not improve from 0.54601
120/120 [==============================] - 1s 7ms/step - loss: 0.4790 - val_loss: 0.5510
Epoch 25/100
113/120 [===========================>..] - ETA: 0s - loss: 0.4710
Epoch 25: val_loss did not improve from 0.54601
120/120 [==============================] - 1s 7ms/step - loss: 0.4759 - val_loss: 0.5605
Epoch 26/100
120/120 [==============================] - ETA: 0s - loss: 0.4684
Epoch 26: val_loss improved from 0.54601 to 0.53681, s

Epoch 92/100
117/120 [============================>.] - ETA: 0s - loss: 0.3840
Epoch 92: val_loss did not improve from 0.50779
120/120 [==============================] - 1s 8ms/step - loss: 0.3825 - val_loss: 0.5305
Epoch 93/100
118/120 [============================>.] - ETA: 0s - loss: 0.3860
Epoch 93: val_loss did not improve from 0.50779
120/120 [==============================] - 1s 8ms/step - loss: 0.3854 - val_loss: 0.5198
Epoch 94/100
119/120 [============================>.] - ETA: 0s - loss: 0.3840
Epoch 94: val_loss did not improve from 0.50779
120/120 [==============================] - 1s 7ms/step - loss: 0.3840 - val_loss: 0.5203
Epoch 95/100
119/120 [============================>.] - ETA: 0s - loss: 0.3815
Epoch 95: val_loss did not improve from 0.50779
120/120 [==============================] - 1s 8ms/step - loss: 0.3813 - val_loss: 0.5106
Epoch 96/100
120/120 [==============================] - ETA: 0s - loss: 0.3762
Epoch 96: val_loss did not improve from 0.50779
120/120 [

...

Epoch 16/100
115/120 [===========================>..] - ETA: 0s - loss: 0.5220
Epoch 16: val_loss did not improve from 0.56833
120/120 [==============================] - 1s 8ms/step - loss: 0.5228 - val_loss: 0.5705
Epoch 17/100
116/120 [============================>.] - ETA: 0s - loss: 0.5070
Epoch 17: val_loss improved from 0.56833 to 0.56696, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold3.hdf5
120/120 [==============================] - 1s 8ms/step - loss: 0.5077 - val_loss: 0.5670
Epoch 18/100
116/120 [============================>.] - ETA: 0s - loss: 0.5013
Epoch 18: val_loss improved from 0.56696 to 0.55953, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\bestModel-fold3.hdf5
120/120 [==============================] - 1s 8ms/step - loss: 0.5035 - val_loss: 0.5595
Epoch 19/100
115/120 [===========================>..] - ETA: 0s - loss: 0.5072
Epoch 19: val

Epoch 45/100
120/120 [==============================] - ETA: 0s - loss: 0.4181
Epoch 45: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 7ms/step - loss: 0.4181 - val_loss: 0.5404
Epoch 46/100
119/120 [============================>.] - ETA: 0s - loss: 0.4253
Epoch 46: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 7ms/step - loss: 0.4250 - val_loss: 0.5388
Epoch 47/100
120/120 [==============================] - ETA: 0s - loss: 0.4214
Epoch 47: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 7ms/step - loss: 0.4214 - val_loss: 0.5391
Epoch 48/100
119/120 [============================>.] - ETA: 0s - loss: 0.4225
Epoch 48: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 7ms/step - loss: 0.4225 - val_loss: 0.5469
Epoch 49/100
115/120 [===========================>..] - ETA: 0s - loss: 0.4211
Epoch 49: val_loss did not improve from 0.53109
120/120 [

Epoch 83/100
117/120 [============================>.] - ETA: 0s - loss: 0.3740
Epoch 83: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 8ms/step - loss: 0.3732 - val_loss: 0.5624
Epoch 84/100
116/120 [============================>.] - ETA: 0s - loss: 0.3737
Epoch 84: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 8ms/step - loss: 0.3720 - val_loss: 0.5662
Epoch 85/100
116/120 [============================>.] - ETA: 0s - loss: 0.3742
Epoch 85: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 8ms/step - loss: 0.3751 - val_loss: 0.5561
Epoch 86/100
119/120 [============================>.] - ETA: 0s - loss: 0.3655
Epoch 86: val_loss did not improve from 0.53109
120/120 [==============================] - 1s 8ms/step - loss: 0.3655 - val_loss: 0.5463
Epoch 87/100
119/120 [============================>.] - ETA: 0s - loss: 0.3861
Epoch 87: val_loss did not improve from 0.53109
120/120 [

120/120 [==============================] - 1s 8ms/step - loss: 0.5273 - val_loss: 0.5301
Epoch 15/100
113/120 [===========================>..] - ETA: 0s - loss: 0.5281
Epoch 15: val_loss did not improve from 0.53008
120/120 [==============================] - 1s 7ms/step - loss: 0.5297 - val_loss: 0.5321
Epoch 16/100
120/120 [==============================] - ETA: 0s - loss: 0.5221
Epoch 16: val_loss did not improve from 0.53008
120/120 [==============================] - 1s 7ms/step - loss: 0.5221 - val_loss: 0.5314
Epoch 17/100
116/120 [============================>.] - ETA: 0s - loss: 0.5168
Epoch 17: val_loss did not improve from 0.53008
120/120 [==============================] - 1s 8ms/step - loss: 0.5209 - val_loss: 0.5373
Epoch 18/100
117/120 [============================>.] - ETA: 0s - loss: 0.5077
Epoch 18: val_loss did not improve from 0.53008
120/120 [==============================] - 1s 8ms/step - loss: 0.5074 - val_loss: 0.5342
Epoch 19/100
118/120 [=========================

Epoch 49/100
114/120 [===========================>..] - ETA: 0s - loss: 0.4204
Epoch 49: val_loss did not improve from 0.50982
120/120 [==============================] - 1s 7ms/step - loss: 0.4231 - val_loss: 0.5262
Epoch 50/100
114/120 [===========================>..] - ETA: 0s - loss: 0.4271
Epoch 50: val_loss did not improve from 0.50982
120/120 [==============================] - 1s 7ms/step - loss: 0.4283 - val_loss: 0.5207
Epoch 51/100
120/120 [==============================] - ETA: 0s - loss: 0.4152
Epoch 51: val_loss did not improve from 0.50982
120/120 [==============================] - 1s 7ms/step - loss: 0.4152 - val_loss: 0.5482
Epoch 52/100
114/120 [===========================>..] - ETA: 0s - loss: 0.4195
Epoch 52: val_loss did not improve from 0.50982
120/120 [==============================] - 1s 7ms/step - loss: 0.4231 - val_loss: 0.5165
Epoch 53/100
115/120 [===========================>..] - ETA: 0s - loss: 0.4190
Epoch 53: val_loss did not improve from 0.50982
120/120 [

## k-fold Training evaluation

In [13]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.785468,0.772456,0.862004,0.810246,0.760691,0.572232
Train,0.904495,0.888980,0.961596,0.925276,0.883718,0.810167


In [14]:
evaluations_df[evaluations_df["Train_Test"] == "Test"]

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.794549,0.787755,"[0.0, 0.0041841004184100415, 0.012552301255230...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9944923, 0.9944923, 0.98615485, 0.9853139, ...",0.872077,0.807531,0.781513,0.589262
3,1,Test,0.792453,0.783673,"[0.0, 0.004201680672268907, 0.0042016806722689...","[0.0, 0.0, 0.0041841004184100415, 0.0041841004...","[1.9888046, 0.98880464, 0.98633575, 0.958263, ...",0.859420,0.806723,0.778243,0.585181
5,2,Test,0.787815,0.754647,"[0.0, 0.004201680672268907, 0.1302521008403361...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9838555, 0.98385555, 0.947379, 0.94682926, ...",0.877613,0.852941,0.722689,0.580576
7,3,Test,0.777311,0.777311,"[0.0, 0.004201680672268907, 0.0504201680672268...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9833007, 0.9833007, 0.9524279, 0.95138997, ...",0.842278,0.777311,0.777311,0.554622
9,4,Test,0.775210,0.758893,"[0.0, 0.004201680672268907, 0.0756302521008403...","[0.0, 0.0, 0.0, 0.008403361344537815, 0.008403...","[1.991195, 0.99119496, 0.95751673, 0.9549976, ...",0.858635,0.806723,0.743697,0.551517


# Independent data

In [15]:
train_features = features
train_labels = labels

In [16]:
##################################################################################
##### read independent data file
##################################################################################
indpe_file_path = os.path.join(input_data_folder, independent_data_file)
indpe_data = pd.read_csv(indpe_file_path, sep='\t', header=None)
indpe_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
indpe_data.head()
    
##################################################################################
##### Create OHE of sequence
##################################################################################
indpe_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in indpe_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
indpe_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in indpe_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(list(indpe_data['OHE_Sequence']))
indpe_labels = np.array(list(indpe_data['label']))
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

input_seq_shape = indpe_features[0].shape

## Using k-fold Models

### Performance of each k-fold model

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.594776,0.242527,0.671541,0.678818,0.578082,0.191901


In [18]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.597551,0.249135,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0019569471624266144, 0.0019569471...","[1.9954306, 0.9954306, 0.9887334, 0.9879417, 0...",0.681172,0.709360,0.575342,0.212055
1,1,Independent,0.612245,0.255396,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0029354207436399216, 0.00293...","[1.9897283, 0.98972833, 0.9892013, 0.98633575,...",0.673633,0.699507,0.594912,0.219882
2,2,Independent,0.550204,0.228125,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0009784735812133072, 0.0009784735...","[1.9900972, 0.99009717, 0.98125285, 0.9802611,...",0.668167,0.719212,0.516634,0.175563
3,3,Independent,0.601633,0.238532,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0019569471624266144, 0.0019569471...","[1.9844738, 0.9844738, 0.9804381, 0.97816294, ...",0.664634,0.640394,0.593933,0.175325
4,4,Independent,0.612245,0.241445,"[0.0, 0.0, 0.0, 0.009852216748768473, 0.009852...","[0.0, 0.0009784735812133072, 0.001956947162426...","[1.9875392, 0.9875392, 0.9834366, 0.9805925, 0...",0.670100,0.625616,0.609589,0.176680


### Mean score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.599184,0.243772,0.682762,0.674877,0.584149,0.193282


### Voting score with k-fold models

In [20]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.598367,0.24515,0.67445,0.684729,0.581213,0.198315


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [21]:
pos_indexes = np.where(indpe_labels==1)[0]
neg_indexes = np.random.permutation(np.where(indpe_labels==0)[0])[0:pos_indexes.shape[0]]
indpe_val_indexes = np.concatenate((pos_indexes, neg_indexes))

In [27]:
model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_accuracy', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

X_features = np.concatenate((train_features, 
                             np.flip(train_features, axis=1)),
                            axis = 0
                           )
Y_labels = np.concatenate((train_labels, 
                           train_labels), 
                          axis = 0)

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(X_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = X_features[index_arr], y = Y_labels[index_arr], batch_size = 16, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features[indpe_val_indexes], indpe_labels[indpe_val_indexes]))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100
296/298 [============================>.] - ETA: 0s - loss: 0.8569 - accuracy: 0.5562
Epoch 1: val_accuracy improved from -inf to 0.59606, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\_fullModel.hdf5
298/298 [==============================] - 4s 9ms/step - loss: 0.8561 - accuracy: 0.5571 - val_loss: 0.8078 - val_accuracy: 0.5961
Epoch 2/100
295/298 [============================>.] - ETA: 0s - loss: 0.7084 - accuracy: 0.6915
Epoch 2: val_accuracy did not improve from 0.59606
298/298 [==============================] - 2s 7ms/step - loss: 0.7081 - accuracy: 0.6914 - val_loss: 0.7918 - val_accuracy: 0.5887
Epoch 3/100
297/298 [============================>.] - ETA: 0s - loss: 0.6491 - accuracy: 0.7197
Epoch 3: val_accuracy improved from 0.59606 to 0.60099, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2_TrainsStrat_Reverse\5fold\models\_fullModel.hdf5
298/298 [==============================] - 2s 8

297/298 [============================>.] - ETA: 0s - loss: 0.3994 - accuracy: 0.8451
Epoch 56: val_accuracy did not improve from 0.63300
298/298 [==============================] - 2s 8ms/step - loss: 0.3995 - accuracy: 0.8447 - val_loss: 0.8181 - val_accuracy: 0.6158
Epoch 57/100
298/298 [==============================] - ETA: 0s - loss: 0.4019 - accuracy: 0.8398
Epoch 57: val_accuracy did not improve from 0.63300
298/298 [==============================] - 2s 8ms/step - loss: 0.4019 - accuracy: 0.8398 - val_loss: 0.8543 - val_accuracy: 0.6305
Epoch 58/100
292/298 [============================>.] - ETA: 0s - loss: 0.4079 - accuracy: 0.8382
Epoch 58: val_accuracy did not improve from 0.63300
298/298 [==============================] - 2s 8ms/step - loss: 0.4085 - accuracy: 0.8382 - val_loss: 0.8190 - val_accuracy: 0.6158
Epoch 59/100
292/298 [============================>.] - ETA: 0s - loss: 0.4040 - accuracy: 0.8403
Epoch 59: val_accuracy did not improve from 0.63300
298/298 [===========

Epoch 85/100
297/298 [============================>.] - ETA: 0s - loss: 0.3911 - accuracy: 0.8415
Epoch 85: val_accuracy did not improve from 0.63793
298/298 [==============================] - 2s 8ms/step - loss: 0.3913 - accuracy: 0.8409 - val_loss: 0.8918 - val_accuracy: 0.6281
Epoch 86/100
297/298 [============================>.] - ETA: 0s - loss: 0.3913 - accuracy: 0.8411
Epoch 86: val_accuracy did not improve from 0.63793
298/298 [==============================] - 2s 8ms/step - loss: 0.3909 - accuracy: 0.8413 - val_loss: 0.9128 - val_accuracy: 0.6133
Epoch 87/100
294/298 [============================>.] - ETA: 0s - loss: 0.3950 - accuracy: 0.8461
Epoch 87: val_accuracy did not improve from 0.63793
298/298 [==============================] - 2s 8ms/step - loss: 0.3974 - accuracy: 0.8447 - val_loss: 0.8612 - val_accuracy: 0.6182
Epoch 88/100
297/298 [============================>.] - ETA: 0s - loss: 0.3901 - accuracy: 0.8430
Epoch 88: val_accuracy did not improve from 0.63793
298/298

In [28]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.667755,0.2713,0.665671,0.596059,0.681996,0.214866
